In [1]:
import pandas as pd
import numpy as np
dic_1w = pd.read_csv("ngram/100k_data.csv",sep=';',names=[0,1])
dic_1w[0] = dic_1w[0].apply(lambda x: str(x).lower())

def candidates(x,check=1):
    
    if check == 1:
        if Is_in_dic(x):
            return []
    
    alphabet = list('abcdefghijklmnopqrstuvwxyz')
    edits = pd.DataFrame(None, columns=['candidates', 'error_type', 'error', 'c'])
    splits = [(x[:i], x[i:]) for i in range(len(x) + 1)]
    dels = []
    inss = []
    # only 1 edit distance from the inputted word
    trans = [[L + R[1] + R[0] + R[2:], 'tran',  R[0] + R[1], R[1] + R[0]] for L, R in splits if len(R) > 1]
    subs = [[L + c + R[1:], 'sub', R[0], L[-1:] + c + R[1:2]] for L, R in splits if R for c in alphabet]

    for L, R in splits:
        if R:
            if L:
                inss.append([L + R[1:], 'ins', R[0], L[-2:] + R[1:2]])
            else:
                inss.append([L + R[1:], 'ins', R[0], '>' + R[1:3]])
        for c in alphabet:
            try:
                dels.append([L + c + R, 'del', c, L[-1]+c])
            except:
                dels.append([L + c + R, 'del', c, '>' + c + R[0:1]])

    for item in dels:
        if Is_in_dic(item[0]):
            edits.loc[len(edits)] = item

    for item in inss:
        if Is_in_dic(item[0]):
            edits.loc[len(edits)] = item

    for item in trans:
        if Is_in_dic(item[0]):
            edits.loc[len(edits)] = item

    for item in subs:
        if Is_in_dic(item[0]):
            edits.loc[len(edits)] = item

    return edits.drop_duplicates(subset='candidates')

def Is_in_dic(word):
    if len(dic_1w[dic_1w[0] == word]) > 0:
        return True
    return False

def prior_calc(w):
    sum_1w = dic_1w[1].sum()
    temp = dic_1w[dic_1w[0] == w]
    try:
        prior = float(temp[1]) / sum_1w
    except:
        prior = 0
    return prior


In [2]:

err_freq = pd.read_csv('ngram/count_1edit.txt', delimiter='\t', encoding='ISO-8859-1', header=None)
alphabet = list('abcdefghijklmnopqrstuvwxyz') + ['>']
del_conf_mat = pd.DataFrame(None, columns=alphabet, index=alphabet)
ins_conf_mat = pd.DataFrame(None, columns=alphabet, index=alphabet)
sub_conf_mat = pd.DataFrame(None, columns=alphabet, index=alphabet)
tran_conf_mat = pd.DataFrame(None, columns=alphabet, index=alphabet)

for row in alphabet:
    for col in alphabet:
        temp1 = err_freq[err_freq[0] == col + '|' + col + row]
        try:
            del_conf_mat.loc[row, col] = int(temp1[1])
        except:
            del_conf_mat.loc[row, col] = 0

        temp2 = err_freq[err_freq[0] == col + row + '|' + col]
        try:
            ins_conf_mat.loc[row, col] = int(temp2[1])
        except:
            ins_conf_mat.loc[row, col] = 0

        temp3 = err_freq[err_freq[0] == row + '|' + col]
        try:
            sub_conf_mat.loc[row, col] = int(temp3[1])
        except:
            sub_conf_mat.loc[row, col] = 0

        temp4 = err_freq[err_freq[0] == row + col + '|' + col + row]
        try:
            tran_conf_mat.loc[row, col] = int(temp4[1])
        except:
            tran_conf_mat.loc[row, col] = 0

del_conf_mat.to_csv('del_mat.csv')
ins_conf_mat.to_csv('ins_mat.csv')
sub_conf_mat.to_csv('sub_mat.csv')
tran_conf_mat.to_csv('tran_mat.csv')

def likelihood_calc(type, error, c,idx):
    
    type=type[idx]
    error = error[idx]
    c = c[idx]
    if type == 'del':
        return del_conf_mat.loc[error, c[0]] / counter_2c(c[-2:])
    elif type == 'ins':
        return ins_conf_mat.loc[error, c[-1]] / counter_2c(c[-2:])
    elif type == 'sub':
        if len(c) == 2 and c[0] == error:
            return sub_conf_mat.loc[error, c[-1]] / counter_2c(c[:2])
        else:
            return sub_conf_mat.loc[error, c[-2]] / counter_2c(c[:2])
    elif type == 'tran':
        return tran_conf_mat.loc[error[0], c[0]] / counter_2c(c)
    
def counter_2c(x):
    return int(dic_1w[dic_1w[0] == x][1])



In [3]:
def search_for_words(word,check =1):
    df = candidates(word,check)
    if len(df) < 1:
        return None
    li_hd = []
    
#     for x in df.candidates:
#         temp = df[df['candidates'] == x]
#         li_hd.append(likelihood_calc(temp.error_type,temp.error,temp.c,temp.index.values[0]))

#     df['likelihood'] = li_hd
#     df['lh_scores'] = df['likelihood'].apply(lambda x: np.log(x))

    prior_score = []
    for x in df.candidates:
        temp = df[df['candidates'] == x]
        prior_score.append(prior_calc(x))

    df['prior_score'] = prior_score
    df['pr_scores'] = df['prior_score'].apply(lambda x: np.log(x))

    df = df.sort_values('pr_scores',ascending=False)
    return df.head(1) if check == 1 else df.head(5)

In [4]:
# import re
# words = str(input())

# k = re.sub(r"[\W]+", ' ', words)
# list_of_unknown_words = []
# for x in k.lower().split(" "):
#     if Is_in_dic(x) == False:
#         list_of_unknown_words.append(x)
# list_of_unknown_words.remove("")
# list_of_unknown_words

In [5]:
# for x in set(list(list_of_unknown_words)):
#     print(x)
#     display(search_for_words(x))

## Bigram

In [6]:
dic_2w = pd.read_csv("ngram/count_2w.txt",sep='\t',names=[0,1])
dic_2w

,0,1
0,0Uplink verified,523545
1,0km to,116103
2,1000s of,939476
3,100s of,539389
4,100th anniversary,158621
...,...,...
286353,zz zzz,203566
286354,zzz zzzz,203590
286355,Über uns,227462
286356,útil esta,140271


In [7]:
def Is_in_dic2(x):
    return dic_2w[dic_2w[0].str.contains(x)] if len(dic_2w[dic_2w[0].str.contains(x)])>0 else []

In [8]:
# import re
# words = str(input())

# k = re.sub(r"[\W]+", ' ', words)
# list_of_unknown_words = []
# split_w = k.lower().split(" ")
# for x in range(len(split_w)):
#     temp = ' '.join(split_w[x:x+2])
    
#     if len(Is_in_dic2(temp))<1:
#         print(temp)
#         list_of_unknown_words.append(temp)
# try:
#     list_of_unknown_words.remove("")
# except:
#     pass


## 1st way - Euclidean Distance

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import euclidean_distances


In [10]:
import warnings
warnings.filterwarnings('ignore')
def eu_distance(x,df,sc_dc=None):

    combined_corpus = list(df[0].values)
    combined_corpus.append(x)
    vectorizer = CountVectorizer()
    features = vectorizer.fit_transform(combined_corpus).todense() 

    score_dict = dict()
    i = 0
    for f in features:
        score_dict[combined_corpus[i]] = euclidean_distances(features[len(features)-1], f)[0][0]
        i+=1
    return score_dict

def prior_calc_bigram(w):
    sum_2w = dic_2w[1].sum()
    temp = dic_2w[dic_2w[0] == w]
    try:
        prior = float(temp[1]) / sum_2w
    except:
        prior = 0
    return prior

from difflib import SequenceMatcher

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

def search_best_candidate(sc_dc,word):
    df = pd.DataFrame(sc_dc, index=['eu_values']).T.reset_index()\
                .rename(columns={'index':'candidates','eu_values':'eu_values'}).sort_values('eu_values')
    df = df[df['eu_values'] > 0]
    df['prior'] = df['candidates'].apply(lambda x: prior_calc_bigram(x))
    df['log_prior'] = df['prior'].apply(lambda x: np.log(x))

    df['lv_dist'] = df['candidates'].apply(lambda x: levenshteinDistance(x,word))
    df = df.sort_values(['lv_dist','log_prior'])
    return df.head(1)

def levenshteinDistance(s1, s2):
    if len(s1) > len(s2):
        s1, s2 = s2, s1

    distances = range(len(s1) + 1)
    for i2, c2 in enumerate(s2):
        distances_ = [i2+1]
        for i1, c1 in enumerate(s1):
            if c1 == c2:
                distances_.append(distances[i1])
            else:
                distances_.append(1 + min((distances[i1], distances[i1 + 1], distances_[-1])))
        distances = distances_
    return distances[-1]



In [11]:
# for x in list_of_unknown_words:
#     i = len(x)
#     print(x)
#     while i>0:
#         temp_word = x[:i]
#         search_word = Is_in_dic2(temp_word)
#         if len(search_word)>0:
#             sc_dc = eu_distance(x,search_word)
#             display(search_best_candidate(sc_dc,x))         
#             break
#         i-=1
        

## 2nd way - Bayesian Inference 2nd Phase

In [12]:
# import tkinter as tk

# def printData(firstName, lastName):
#     print(firstName)
#     print(lastName)
#     root.destroy()

# def get_input():

#     firstName = entry1.get()
#     lastName = entry2.get()
#     printData(firstName, lastName)


# root = tk.Tk()
# #Label 1
# label1 = tk.Label(root,text = 'First Name')
# label1.pack()
# label1.config(justify = 'center')

# entry1 = tk.Entry(root, width = 30)
# entry1.pack()

# label3 = tk.Label(root, text="Last Name")
# label3.pack()
# label1.config(justify = 'center')

# entry2 = tk.Entry(root, width = 30)
# entry2.pack()

# button1 = tk.Button(root, text = 'submit')
# button1.pack() 
# button1.config(command = get_input)



In [13]:
# window = tk.Tk()
# window.geometry("700x350")
# text_box = tk.Text(window)
# text_box.pack()
# text_box.insert(tk.INSERT, "Maximum word count is 500 words")
# text_box.tag_add("start", "1.0","1.7")
# text_box.tag_configure("start", background="OliveDrab1", foreground="black")
# text_box.pack()

# button1 = tk.Button(window, text = 'submit')
# button1.pack() 
# button1.config()


# window.mainloop()

In [14]:
# from tkinter import *
# import webbrowser
# win = Tk()
# win.geometry("700x350")

# def callback(event):
#     book = event.widget
#     text = book.cget("text")
#     book.config(bg='blue')
#     print(text)
#     win.destroy()

# # Create a Label Widget
# label= Label(win, text= "Welcome to TutorialsPoint", cursor= "hand2", foreground= "green", font= ('Aerial 18'))
# label.pack(pady= 30)

# # Define the URL to open
# url= 'https://www.tutorialspoint.com/'

# # Bind the label with the URL to open in a new tab
# label.bind("<Button-1>", callback)


# win.mainloop()

## Asking and getting the text

In [15]:
# from tkinter import *

# def printData(firstName, lastName):
#     print(firstName)
#     print(lastName)
    

# def get_input():

#     firstName = entry1.get("1.0",END)
# #     lastName = entry2.get()
#     print(firstName)
#     root.destroy()


# root = tk.Tk()
# root.geometry("700x700")

# #Label 1
# label1 = tk.Label(root,text = 'First Name')
# label1.pack()
# label1.config(justify = 'center')

# entry1 = tk.Text(root)
# entry1.pack()


# button1 = tk.Button(root, text = 'submit')
# button1.pack() 
# button1.config(command = get_input)

# root.mainloop()

## Serious

In [16]:
from tkinter import *
import re
class saveInput:
    def __init__(self):
        self.words = ''
        
def get_input():
    
    firstName = entry1.get("1.0",END)
    saveWords.words = firstName
    root.destroy()
    
def unigram_execution(words):
    
    k = re.sub(r"[\W]+", ' ', words)
    list_of_unknown_words = dict()
    stat_words = 0
    split_w = k.lower().split(" ")
    for x in range(len(split_w)):
#         temp = ' '.join(split_w[x:x+2])
        temp = stat_words+len(split_w[x])
        temp_w = split_w[x]
        if temp_w in list_of_unknown_words.keys():
            n_w = [x for x in list_of_unknown_words.keys() if temp_w in x][-1].split(" ")
            temp_w = temp_w + ' '+  str(int(n_w[1])+1) if len(n_w) >= 2 else temp_w + " 1"
        list_of_unknown_words[temp_w] = {'start': stat_words,'end':temp}
        stat_words = temp+1
        
    
    return list_of_unknown_words

def bigram_execution(words):
    import re

#     k = re.sub(r"[\W]+", ' ', words)
    
    list_of_unknown_words = dict()
    split_w = words.lower().split(" ")
    start_words = 0
    w_count = 0
    for x in range(len(split_w)):
        
        temp = split_w[x:x+2]
        j_temp = ' '.join(temp)

        if j_temp in list_of_unknown_words.keys():
            n_w = [x for x in list_of_unknown_words.keys() if j_temp in x][-1].split("_")
            print(n_w,j_temp)
            j_temp = j_temp + '_'+  str(int(n_w[1])+1) if len(n_w) >= 2 else j_temp + "_1"
            
        if len(Is_in_dic2(j_temp))<1:
            
            list_of_unknown_words[j_temp] = {'start':start_words,'end':(start_words+len(temp[0])+len(temp[1])+1),
                                             'array':[w_count,w_count+1]}
            
        start_words += len(temp[0])+1
        w_count+=1
        
    try:
        list_of_unknown_words.remove("")
    except:
        pass
    
    return list_of_unknown_words


def get_words(data):
    import pandas as pd
    
    df_words = pd.DataFrame(data).T
    df_words = df_words.sort_values("start").reset_index()
    return df_words

def add_highlighter(df):
    
    inputted_words.tag_add("start", '1.'+str(df[1]),'1.'+str(df[2]))
    inputted_words.tag_config("start", background= "black", foreground= "white")

def generate_words_recommendation(p_words):
    candidates_words_recommendation = pd.DataFrame()
    for x in p_words['index']:
        temp = x
        print(temp)
        i = len(temp)
        while i>0:
            temp_word = temp[:i]
            search_word = Is_in_dic2(temp_word)
            if len(search_word)>0:
                sc_dc = eu_distance(x,search_word)
                cd_res = search_best_candidate(sc_dc,x)
                break
            i-=1
        candidates_words_recommendation = candidates_words_recommendation.append((cd_res))
    return candidates_words_recommendation
#     print(df[0],df[1],df[2])

In [17]:
saveWords = saveInput()

root = Tk()
root.geometry("700x700")

#Label 1
label1 = Label(root,text = 'Maximum 500 words')
label1.pack()
label1.config(justify = 'center')

entry1 = Text(root)
entry1.pack()


button1 = Button(root, text = 'submit')
button1.pack() 
button1.config(command = get_input)

root.mainloop()

In [18]:
## Process all words

k = re.sub(r"[\W]+", ' ', saveWords.words)
saveWords.words = k


# Process the save words
data = bigram_execution(saveWords.words)
p_words = get_words(data)
candidates_words_recommendation = generate_words_recommendation(p_words)
p_words['candidate_words'] = candidates_words_recommendation['candidates'].values
p_words['combined_words'] = p_words['index'].astype(str) + ' = ' + p_words['candidate_words'].astype(str)
splitted_inputted_words = saveWords.words.split(" ")

for x,y in zip(p_words.array,p_words.candidate_words):
    temp_split = y.split(" ")
    for z,i in zip(x,temp_split):
        splitted_inputted_words[z] = i
try:
    splitted_inputted_words.remove("")
except:
    pass

## Bigram process ended

## Unigram started
unigram_execution = pd.DataFrame()
i = 0
for x in splitted_inputted_words:
    temp_candidates = search_for_words(str(x).lower(),1)
    if temp_candidates is not None:
        temp_candidates['words'] = str(x)
        temp_candidates['array'] = i
        
    unigram_execution = unigram_execution.append(temp_candidates)
    i+=1



his intetion
intetion during
patent he
such experiments
experiments nor
any partculars
partculars respecting
respecting them
whether more
of overcming
overcming the
the difficuties
difficuties experienced
experienced was
was sugested
sugested and
tried or
nor if
was attemted
attemted what


In [19]:
candidates_words_recommendation

,candidates,eu_values,prior,log_prior,lv_dist
3,his intention,1.414214,1.352431e-06,-13.513606,1
773,intention and,2.000000,5.582964e-07,-14.398376,6
4,patent in,1.414214,4.932791e-07,-14.522191,2
4,such equipment,1.414214,8.187241e-07,-14.015519,6
3,experiments for,1.414214,5.148940e-07,-14.479305,1
1,any particular,1.414214,1.175399e-05,-11.351318,2
930,particular section,2.000000,4.847154e-07,-14.539704,7
0,respecting the,1.414214,2.969495e-06,-12.727119,1
0,whether my,1.414214,5.952196e-07,-14.334335,3
0,of overcoming,1.414214,6.010925e-07,-14.324517,1


In [20]:
root = Tk()
# root.geometry("700x700")
root.attributes('-fullscreen', True)
#Label 1
label1 = Label(root,text = 'Maximum 500 words')
# label1.pack()
label1.config(justify = 'center')

inputted_words = Text(root)
inputted_words.insert(INSERT,saveWords.words + "\n\n\n")
p_words.apply(add_highlighter,axis=1)


# Insert into it
inputted_words.insert(INSERT,"Processed words(aft bigram): " + ' '.join(splitted_inputted_words))

inputted_words_2 = Text(root)
inputted_words_2.insert(INSERT,"\n".join(p_words['combined_words'].values))
inputted_words_2.insert(END,"\n")

if len(unigram_execution)> 0:
    for x,y in zip(unigram_execution.candidates,unigram_execution.array):
        splitted_inputted_words[y] = str(x)
    unigram_execution['combined_words'] = unigram_execution['words'] + ' = ' + unigram_execution['candidates']
    inputted_words.insert(INSERT,"\n\nProcessed words(aft unigram): " + ' '.join(splitted_inputted_words))
    inputted_words_2.insert(INSERT,"\n".join(unigram_execution['combined_words'].values))
    inputted_words_2.insert(END,"\n")


# Label 2

inputted_words_2.config(state=DISABLED)
# inputted_words_2.pack()

inputted_words.config(state=DISABLED)
# inputted_words.pack()

inputted_words_2.grid(column=2,row=0, padx=100, pady=10)  #,ipadx=0,padx=0,sticky=E+W)
inputted_words.grid(column=1,row=0)

In [ ]:
def find_unigram_candidate(splitted_inputted_words = splitted_inputted_words):
    selected_words = Lb1.curselection()[0]
    df = search_for_words(str(splitted_inputted_words[selected_words]),0)
    df['combined_candidates_score'] = df.candidates.astype(str) +' = '+df.pr_scores.apply(lambda x: round(x,2)).astype(str)
    inputted_words_3.delete(1.0,"end")
    if len(df)>0:
        inputted_words_3.insert(INSERT,"Recommendation for {} \n\n".format(splitted_inputted_words[selected_words]) + 
                                    "\n".join(df['combined_candidates_score'].values))
    else:
        inputted_words_3.insert(INSERT,"No Recommendation for this word {}".format(splitted_inputted_words[selected_words]))
    
inputted_words_3 = Text(root)
inputted_words_3.grid(column=2,row=1, padx=100, pady=10)
    
## Listbox ##
Lb1 = Listbox(root,width=len(splitted_inputted_words))
i = 0
for x in splitted_inputted_words:
    
    Lb1.insert(i+1, x)
    i+=1

Lb1.grid(column=1,row=1)

button1 = Button(root, text = 'Find candidates')
button1.grid(column=1,row=2) 
button1.config(command = find_unigram_candidate)

## Listbox ##

root.mainloop()

In [ ]:
# unigram_execution = pd.DataFrame()
# i = 0
# for x in splitted_inputted_words:
#     temp_candidates = search_for_words(str(x),1)
#     if temp_candidates is not None:
#         temp_candidates['words'] = str(x)
#         temp_candidates['array'] = i
        
#     unigram_execution = unigram_execution.append(temp_candidates)
#     i+=1

# for x,y in zip(unigram_execution.candidates,unigram_execution.array):
#     splitted_inputted_words[y] = str(x)
unigram_execution

In [ ]:
unigram_execution['combined_words'] = unigram_execution['words'] + ' = ' + unigram_execution['candidates']
unigram_execution